In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src')))
sys.path.append(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src/scripts/demo_202107'))

CONNECT_INDY = False
CONNECT_MOBILE = False
USE_SDK = True

IP_CUR = "192.168.0.10"# get_ip_address()
MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"

print("Current PC IP: {}".format(IP_CUR))
print("Mobile ROB IP: {}".format(MOBILE_IP))

Current PC IP: 192.168.0.10
Mobile ROB IP: 192.168.0.102


In [2]:
# add camera geometry
def add_cam(gscene, tool_link="indy0_tcp"):
    gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="cam", link_name=tool_link,
                       dims=(0.061, 0.061, 0.026), center=(-0.0785, 0, 0.013), rpy=(0, 0, 0),
                       color=(0.8, 0.8, 0.8, 0.5), display=True, fixed=True, collision=False)

    gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="cam_col", link_name=tool_link,
                       dims=(0.081, 0.081, 0.046), center=(-0.0785, 0, 0.013), rpy=(0, 0, 0),
                       color=(0.8, 0.8, 0.8, 0.2), display=True, fixed=True, collision=True)

    viewpoint = gscene.create_safe(gtype=GEOTYPE.SPHERE, name="viewpoint", link_name=tool_link,
                                   dims=(0.01, 0.01, 0.01), center=(-0.013, 0, 0), rpy=(0, 0, -np.pi / 2),
                                   color=(1, 0, 0, 0.3), display=True, fixed=True, collision=False, parent="cam")

    gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="body", link_name=tool_link,
                       dims=(0.067, 0.067, 0.0335), center=(-0.0785, 0, -0.01675), rpy=(0, 0, 0),
                       color=(0.8, 0.8, 0.8, 1), display=True, fixed=True, collision=False)

    gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="body_col", link_name=tool_link,
                       dims=(0.087, 0.087, 0.0535), center=(-0.0785, 0, -0.01675), rpy=(0, 0, 0),
                       color=(0.8, 0.8, 0.8, 0.2), display=True, fixed=True, collision=True)

    gscene.create_safe(gtype=GEOTYPE.SPHERE, name="backhead", link_name=tool_link,
                       dims=(0.067, 0.067, 0.067), center=(-0.0785, 0, -0.0335), rpy=(0, 0, 0),
                       color=(0.8, 0.8, 0.8, 1), display=True, fixed=True, collision=False)

    gscene.create_safe(gtype=GEOTYPE.SPHERE, name="backhead_col", link_name=tool_link,
                       dims=(0.087, 0.087, 0.087), center=(-0.0785, 0, -0.0335), rpy=(0, 0, 0),
                       color=(0.8, 0.8, 0.8, 0.2), display=True, fixed=True, collision=True)
    return viewpoint

In [3]:
INDY_BASE_OFFSET = (0.172,0,0.439)
INDY_BASE_RPY = (0,0,0)
TOOL_NAME = "brush_face"
WALL_THICKNESS = 0.01
CLEARANCE = 0.001
WS_HEIGHT = 1.6
COL_COLOR = (1,1,1,0.2)
    
# if EXP_SCENARIO == ExpType.REMOVE_OBS: ## Obstacle removing
#     BAG_COUNT = 5
#     CLOCK_COUNT = 0
#     TARGET_COUNT = 5
#     LOG_FORCE = False
# else: ## Contact 
#     BAG_COUNT = 3
#     CLOCK_COUNT = 3
#     TARGET_COUNT = 5
#     LOG_FORCE = True

from pkg.controller.combined_robot import *
from pkg.project_config import *

if not CONNECT_INDY:
    indy_7dof_client.kiro_tool.OFFLINE_MODE = True
kiro_udp_client.KIRO_UDP_OFFLINE_DEBUG = not CONNECT_MOBILE

mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                "{}/{}".format(MOBILE_IP, IP_CUR))
robot_config = RobotConfig(1, RobotType.indy7, 
                           (INDY_BASE_OFFSET, INDY_BASE_RPY),
                INDY_IP, root_on="kmb0_platform", 
                           specs={"no_sdk":True} if not USE_SDK else {})

ROBOT_TYPE = robot_config.type
MOBILE_NAME = mobile_config.get_indexed_name()
ROBOT_NAME = robot_config.get_indexed_name()
crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config]
              , connection_list=[True, CONNECT_INDY])

connection command:
kmb0: True
indy1: False


In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
from pkg.planning.scene import PlanningScene
from pkg.geometry.geometry import GeometryItem
from pkg.geometry.geotype import GEOTYPE

s_builder = SceneBuilder(None)
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(
    color=COL_COLOR, display=True, collision=True)

gscene.set_workspace_boundary(
    -1, 4, -2.5, 2.5, -CLEARANCE, WS_HEIGHT, thickness=WALL_THICKNESS)

viewpoint = add_cam(gscene, tool_link="indy1_tcp")

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


### MultiICP detector

In [5]:
from pkg.detector.multiICP.multiICP import MultiICP, MultiICP_Obj
from pkg.detector.multiICP.config import *
from pkg.detector.camera.realsense import RealSense
from pkg.detector.detector_interface import DetectionLevel

import open3d as o3d
import numpy as np

CONNECT_CAM = True

In [6]:
realsense = RealSense()

# from demo_utils.data_reconstructed_camera import DataRecontructedCamera
# dcam = DataRecontructedCamera(crob, viewpoint)
# if not CONNECT_CAM:
#     dcam.initialize()
    
if CONNECT_CAM:
    micp = MultiICP(realsense)
    micp.initialize()
#     dcam.ready_saving(*realsense.get_config())
#     cam_pose = viewpoint.get_tf(VIEW_POSE_EXT)
else:
    # use manually given camera configs
    config_list, img_dim = load_pickle(RNB_PLANNING_DIR+"release/multiICP_data/cam_configs.pkl")
    micp.initialize(config_list, img_dim)
    micp = MultiICP(dcam)
    micp.initialize()

Start streaming
Initialize Done


### Shared Detector

In [7]:
from pkg.detector.multiICP.shared_detector import SharedDetectorGen
sd = SharedDetectorGen(tuple(reversed(micp.dsize))+(3,))()
sd.init()

### set ICP config

In [8]:
# Load config file of object information
obj_info_dict = get_obj_info()

In [9]:
# initial guess는 실험하는거 보면서 수정?
micp_suitcase = MultiICP_Obj(obj_info_dict["suitcase"], None,
                        OffsetOnModelCoord("suitcase", R=np.identity(3), offset=(0.,0.,0.)))


micp_clock = MultiICP_Obj(obj_info_dict["clock"], None,
                        OffsetOnModelCoord("clock", R=np.identity(3), offset=(0.,0.,0.)))


micp_table = MultiICP_Obj(obj_info_dict["dining table"], None,
                        OffsetOnModelCoord("dining table", R=np.identity(3), offset=(0.,0.,0.)))

In [10]:
micp_dict = {"suitcase": micp_suitcase, "clock": micp_clock, "dining table": micp_table}
# micp_dict = {"clock": micp_clock}
# micp_dict = {"dining table": micp_table}
# micp_dict = {"suitcase": micp_suitcase}

In [11]:
# set config information for micp
micp.set_config(micp_dict, sd, crob, viewpoint)

### Detect

In [ ]:
pose_dict = micp.detect(visualize=True)

In [17]:
pose_dict = micp.detect(name_mask=["suitcase", "clock", "dining table"])

name_mask is ['suitcase', 'clock', 'dining table']
suitcase not detected
dining table not detected
clock not detected


### Update Object to scene

In [5]:
SEP_DISTANCE = 0.4
TABLE_HEIGHT = 0.734

In [6]:
# add object
def add_carrier(gscene, name, carrier_center, carrier_rpy):
    obj_vis = gscene.create_safe(GEOTYPE.MESH, name, link_name="base_link",
                                 dims=(0.1,0.1,0.1), center=carrier_center, rpy=carrier_rpy,
                                 color=(0.8,0.8,0.8,1), display=True, fixed=False, collision=True,
                                 uri="package://my_mesh/meshes/stl/carrier.STL", scale=(1e-3,1e-3,1e-3))
    return obj_vis

def add_clock(gscene, name, clock_center, clock_rpy):
    obj_vis = gscene.create_safe(GEOTYPE.MESH, name, link_name="base_link",
                                 dims=(0.1,0.1,0.1), center=clock_center, rpy=clock_rpy,
                                 color=(0.8,0.8,0.8,1), display=True, fixed=False, collision=True,
                                 uri="package://my_mesh/meshes/stl/tableclock.STL", scale=(1e-3,1e-3,1e-3))
    return obj_vis

def add_table(gscene, name, clock_center, clock_rpy):
    obj_vis = gscene.create_safe(GEOTYPE.MESH, name, link_name="base_link",
                                 dims=(0.1,0.1,0.1), center=clock_center, rpy=clock_rpy,
                                 color=(0.8,0.8,0.8,1), display=True, fixed=True, collision=True,
                                 uri="package://my_mesh/meshes/stl/table.STL", scale=(1e-3,1e-3,1e-3))
    return obj_vis


# move object(pose update)
def move_carrier(gscene, name, carrier_center, carrier_rpy):
    obj_vis = gscene.NAME_DICT[name]
    obj_vis.set_offset_tf(center=carrier_center, orientation_mat=Rot_rpy(carrier_rpy))
    gscene.update_markers_all()
    
def move_clock(gscene, name, clock_center, clock_rpy):
    obj_vis = gscene.NAME_DICT[name]
    obj_vis.set_offset_tf(center=clock_center, orientation_mat=Rot_rpy(clock_rpy))
    gscene.update_markers_all()
    
def move_table(gscene, name, table_center, table_rpy):
    obj_vis = gscene.NAME_DICT[name]
    obj_vis.set_offset_tf(center=table_center, orientation_mat=Rot_rpy(table_rpy))
    gscene.update_markers_all()
    

def pose_refine(obj_type, T, table_height = 0.734):
    T_new = align_z(T)
    
    center = T_new[:3,3]
    if obj_type == "suitcase" or "dining table":
        center[2] = 0
    elif obj_type == "clock":
        center[2] = TABLE_HEIGHT
    rpy = Rot2rpy(T_new[:3,:3])
    
    return center, rpy

In [7]:
# add & update for suitcase, clock
def add_update_object(gscene, obj_type, pose_dict, separte_dist = 0.5):
    # check num of object in the scene
    obj_count = 0
    for i in range(10):
        name = "{}_{:01}".format(obj_type, i)
        if name in gscene.NAME_DICT.keys():
            obj_count += 1
    print("Total {} {} in the scene".format(obj_count, obj_type))

    if obj_count > 0: # If there are objects, add and update objects from detected result
        not_update_list = []
        for name in pose_dict.keys():
            if "_" in name:
                name_cat = name.split("_")[0]
            else:
                name_cat = name
            if name_cat != "suitcase" and name_cat !="clock":
                pass
            else:
                T = pose_dict[name]
                center, rpy = pose_refine(obj_type, T)

                check_update = False
                for i in range(obj_count):
                    obj_name = "{}_{:01}".format(obj_type, i)
                    T_ = gscene.NAME_DICT[obj_name].get_tf(crob.get_real_robot_pose())
                    center_, rpy_ = pose_refine(obj_type, T_)

                    if np.linalg.norm(center-center_) < separte_dist: # consider same object
                        move_carrier(gscene, obj_name, center, rpy)
                        print("Update existing {} in the scene".format(obj_type))
                        check_update = True
                if not check_update:
                    not_update_list.append(name)

        # treat new object
        for name in not_update_list:
            T = pose_dict[name]
            center, rpy = pose_refine(obj_type, T)
            new_obj_name = "{}_{:01}".format(obj_type, obj_count)
            add_carrier(gscene, new_obj_name, center, rpy)
            print("Add new {} in the scene".format(obj_type)) 

    else: # At first, there is no object, add all detected object in the scene
        count_tmp = 0
        for name in pose_dict.keys():
            if "_" in name:
                name_cat = name.split("_")[0]
            else:
                name_cat = name
            if name_cat != "suitcase" and name_cat !="clock":
                pass
            else:
                T = pose_dict[name]
                center, rpy = pose_refine(obj_type, T)

            new_obj_name = "{}_{:01}".format(obj_type, count_tmp)
            add_carrier(gscene, new_obj_name, center, rpy)
            count_tmp += 1
            print("Add new {} in the scene".format(obj_type))

In [ ]:
# add table
center, rpy = pose_refine("dining table", pose_dict["dining table"])
add_table(gscene, "table", center, rpy)

In [ ]:
# add or update suitcase
add_update_object(gscene, "suitcase", pose_dict)

In [ ]:
# add or update clock
add_update_object(gscene, "clock", pose_dict)

### Test code for multiple instance detection
* MultiICP Detector 이전까지 실행시키고 아래의 코드 실행
* Separte distance는 수동으로 조절

In [8]:
# Origianlly, 4 suitcases are existed in scene
c10 = add_carrier(gscene, "suitcase_0", (1.,1.,0), (0,0,0))
c11 = add_carrier(gscene, "suitcase_1", (1.,2.,0), (0,0,0))
c12 = add_carrier(gscene, "suitcase_2", (2.,1.,0), (0,0,0))
c13 = add_carrier(gscene, "suitcase_3", (2.,2.,0), (0,0,0))

In [17]:
# Detection result, but T1, T2 are already detected objects so, these are in the scene
# T3 is newly detected object so this is not in the scene
# Therefore, 1 and 2 should be updated, but 3 should be added

T1 = SE3(Rot_axis(3, np.pi/10), (1.1,1.18,0.3))
T2 = SE3(np.identity(3), (2.1,1.88,-0.1))
T3 = SE3(Rot_axis(1, -np.pi/5) , (3.1,1.9,-0.2))
pose_dict = {"suitcase_0":T1, "suitcase_1":T2, "suitcase_2":T3}

In [18]:
add_update_object(gscene, "suitcase", pose_dict)

Total 5 suitcase in the scene
Update existing suitcase in the scene
Update existing suitcase in the scene
Update existing suitcase in the scene
